# Building a Churn Prediction Model with Keras

In [1]:
import datetime, time
import pandas as pd
import numpy as np
import tensorflow as tf
import random, os
import sys, json, joblib
import sklearn.metrics as metrics
from pathlib import Path
from sklearn.metrics import average_precision_score, accuracy_score, precision_recall_curve, f1_score

ROOT = Path.cwd()
SRC_DIR = ROOT/"src"
MODELS_DIR = ROOT/"models"

import warnings
warnings.filterwarnings("ignore")

for p in (SRC_DIR, MODELS_DIR, ROOT):
    p_str = str(p.resolve())
    if p_str not in sys.path:
        sys.path.insert(0, p_str)

from src.churn_predictor import ChurnPredictor
from src.preprocessing import build_preprocessor, PreprocessConfig
from config import Config
from models.model import build_gb, calibrate_prefit

import shutil
from tensorflow.keras.callbacks import TensorBoard

In [2]:
os.environ["PYTHONHASHSEED"] = str(Config.SEED)
random.seed(Config.SEED)
np.random.seed(Config.SEED)
tf.keras.utils.set_random_seed(Config.SEED)
#tf.config.experimental.enable_op_determinism()

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# Tensorboard callback
log_dir = Path('logs')/("integrate_run"+datetime.datetime.now().strftime('%d%m%Y-%H%M%S'))
tb_cb = TensorBoard(log_dir=str(log_dir), histogram_freq=1, write_graph=True, write_images=True)
shutil.rmtree('logs/', ignore_errors=True) # Turn off to see different logs

In [5]:
# Load dataframe+split
df = pd.read_csv(Config.DATA_URL)
df.head()

,gender,SeniorCitizen,Dependents,tenure,PhoneService,MultipleLines,InternetService,Contract,MonthlyCharges,Churn
0,Female,0,No,1,No,No,DSL,Month-to-month,25,Yes
1,Male,0,No,41,Yes,No,DSL,One year,25,No
2,Female,0,Yes,52,Yes,No,DSL,Month-to-month,19,No
3,Female,0,No,1,Yes,No,DSL,One year,76,Yes
4,Male,0,No,67,Yes,No,Fiber optic,Month-to-month,51,No


In [6]:
cp = ChurnPredictor(expect_numeric=False) # False if str data still exist
df["Churn"] = df["Churn"].map({"Yes":1,"No":0})

X_train_full, X_valid, X_test, y_train_full, y_valid, y_test = cp.split(
    df, y_col='Churn', test_size=Config.TEST_SIZE, val_size=Config.VAL_SIZE, seed=Config.SEED)

In [7]:
# Training 
cfg = PreprocessConfig(expect_numeric=False)
preproc, get_names = build_preprocessor(X_train_full, cfg)
preproc.fit(X_train_full)
feature_names = get_names()
print(feature_names)

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'gender_Male', 'Dependents_Yes', 'PhoneService_Yes', 'MultipleLines_Yes', 'InternetService_Fiber optic', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year']


In [8]:
# Transform all splits
X_tr = preproc.transform(X_train_full)
X_va = preproc.transform(X_valid)
X_te = preproc.transform(X_test)

In [10]:
# Tuning
best_hp = cp.tune(X_tr, y_train_full, X_va, y_valid, project_name='krs_hyperband')
best_params = cp.pick_best_params(min_val_acc=Config.MIN_VAL_ACCURACY)
print("Best parameters: ", best_params)

Trial 90 Complete [00h 00m 41s]
val_auprc: 0.5789346893628439

Best val_auprc So Far: 0.5934504270553589
Total elapsed time: 00h 26m 34s
Best parameters:  {'units1': 32, 'units2': 48, 'units3': 8, 'lr': 0.001675450818342236, 'dropout': 0.2}


In [11]:
# Checkpoint: Save for fit
Path("datasets").mkdir(parents=True, exist_ok=True)

payload = {
    "best_params": best_params,
    "seed": Config.SEED,
    "feature_count": int(X_tr.shape[1]),
    "saved_at": time.strftime('%d-%m-%Y-%H:%M:%S')
}

with open(Path("datasets")/"best_params.json", 'w', encoding="utf-8") as f:
    json.dump(payload, f, indent=2)

with open(Path("datasets")/"best_hp.json", 'w', encoding="utf-8") as f:
    json.dump(best_hp.values, f, indent=2)

In [12]:
cp.preprocess_fit(X_train_full)

In [13]:
# Save fit
joblib.dump(cp.preprocessor, "models/preprocessor.joblib")

['models/preprocessor.joblib']

In [10]:
# Final fit (without tensorboard)
class_w = cp.compute_class_weight(y_train_full)
hist = cp.fit_final(X_tr, y_train_full, X_va, y_valid, 
                    best_params, epochs=Config.EPOCHS, batch_size=Config.BATCH_SIZE, class_weights=class_w)

In [15]:
# Evaluate
metrics_nn = cp.evaluate(X_te, y_test)
print(f"(Keras NN)Test AUPRC: {metrics_nn['auprc']:.4f} | Test accuracy: {metrics_nn['accuracy']:.4f}")

(Keras NN)Test AUPRC: 0.5770 | Test accuracy: 0.7795


In [16]:
model = cp.model
y_prob = model.predict(X_te)
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
f1 = 2*(precision*recall)/(precision+recall+1e-8)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [17]:
best_idx = np.argmax(f1)
best_thr = thresholds[best_idx]

In [18]:
y_pred_f1 = (y_prob >= best_thr).astype(int)

In [19]:
acc_f1 = accuracy_score(y_test, y_pred_f1)
print(f"Test accuracy (f1 threshold): {acc_f1:.4f}")

Test accuracy (f1 threshold): 0.7700


In [9]:
# Reload best parameters for fitting (fresh session)
'''
Redo preprocessing and transforms, records are saved after tuning
'''
with open(Path("datasets")/"best_params.json", 'r', encoding="utf-8") as f:
    saved = json.load(f)
best_params = saved["best_params"]

In [10]:
# Final fit (with tensorboard)
#class_w = cp.compute_class_weight(y_train_full)
hist = cp.fit_with_tensorboard(X_tr, y_train_full, X_va, y_valid, 
                               best_params, epochs=Config.EPOCHS, batch_size=Config.BATCH_SIZE, class_weights=None, tb_cb=tb_cb)

## Use Keras probabilities to train a calibrated GB

In [12]:
# Stacking
nn_tr = cp.predict_proba(X_tr).reshape(-1)
nn_va = cp.predict_proba(X_va).reshape(-1)
nn_te = cp.predict_proba(X_te).reshape(-1)

X_tr_st = np.column_stack([X_tr, nn_tr])
X_va_st = np.column_stack([X_va, nn_va])
X_te_st = np.column_stack([X_te, nn_te])

In [13]:
# Fit on GB
gb = build_gb(random_state=Config.SEED)
gb.fit(X_tr_st, y_train_full)

GradientBoostingClassifier(random_state=42)

In [14]:
# Calibrate on val
gb_cal = calibrate_prefit(gb, X_va_st, y_valid, method='isotonic')
gb_cal.fit(X_va_st, y_valid)

CalibratedClassifierCV(cv='prefit',
                       estimator=GradientBoostingClassifier(random_state=42),
                       method='isotonic')

In [15]:
# Pick threshold on validation
proba_va = gb_cal.predict_proba(X_va_st)[:,1]
prec, rec, thr = precision_recall_curve(y_valid, proba_va)
preds_va = (proba_va[:, None] >= thr[None, :]).astype(int)

In [16]:
# Vectorised metrics
auprc_curve = np.array([
    average_precision_score(y_valid, preds_va[:, i]) for i in range(preds_va.shape[1])
])
acc_curve = np.mean(preds_va==y_valid[:, None], axis=0)
f1_curve = np.array([f1_score(y_valid, preds_va[:, i]) for i in range(preds_va.shape[1])])

In [17]:
# Choose threshold
target_auprc = 0.6
feasible = auprc_curve >= target_auprc
if feasible.any():
    best_idx_acc = np.argmax(acc_curve*feasible)
else:
    best_idx_acc = np.argmax(acc_curve)
    
best_thr_acc = float(thr[best_idx_acc])
best_idx_f1 = np.argmax(f1_curve)
best_thr_f1 = float(thr[best_idx_f1])

print(f"[VAL] Best-ACC threshold:{best_thr_acc:.3f} | " f"ACC={acc_curve[best_idx_acc]:.4f} | " f"AUPRC={auprc_curve[best_idx_acc]:.4f}")
print(f"[VAL] Best-F1 threshold:{best_thr_f1:.3f} | " f"F1={f1_curve[best_idx_f1]:.4f} | " 
      f"AUPRC={auprc_curve[best_idx_f1]:.4f} | " f"ACC={acc_curve[best_idx_f1]:.4f}")

[VAL] Best-ACC threshold:0.500 | ACC=0.7909 | AUPRC=0.4457
[VAL] Best-F1 threshold:0.309 | F1=0.6059 | AUPRC=0.4533 | ACC=0.7588


In [18]:
# Evaluate on test with best threshold
proba_te = gb_cal.predict_proba(X_te_st)[:,1]
pred_te_f1 = (proba_te>=best_thr_f1).astype(int)
pred_te_acc = (proba_te>=best_thr_acc).astype(int)

auprc_te = average_precision_score(y_test, proba_te)
acc_te_f1 = accuracy_score(y_test, pred_te_f1)
f1_te_f1 = f1_score(y_test, pred_te_f1)
acc_te_acc = accuracy_score(y_test, pred_te_acc)
f1_te_acc = f1_score(y_test, pred_te_acc)
print(f"AUPRC: {auprc_te:.4f}")
print(f"[TEST-acc_thr] Accuracy: {acc_te_acc:.4f} | F1: {f1_te_acc:.4f}")
print(f"[TEST-f1_thr] Accuracy: {acc_te_f1:.4f} | F1: {f1_te_f1:.4f}")

AUPRC: 0.5836
[TEST-acc_thr] Accuracy: 0.7818 | F1: 0.5339
[TEST-f1_thr] Accuracy: 0.7525 | F1: 0.5879


In [19]:
# Save best threshold
thresholds = {
        "f1": float(best_thr_f1),
        "acc": float(best_thr_acc)
}

with open("datasets/best_threshold.json", 'w') as f:
    json.dump(thresholds, f, indent=4)

In [21]:
model = cp.model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 48)             │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 48)             │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,549 (29.49 KB)

 Trainable params: 2,457 (9.60 KB)

 Non-trainable params: 176 (704.00 B)

 Optimizer params: 4,916 (19.21 KB)

In [11]:
# Load Tensorboard in Jupyter
%reload_ext tensorboard
%tensorboard --logdir logs 

### Optional Saves

In [22]:
# Save splits
np.save("datasets/X_train_preproc.npy", X_tr)
np.save("datasets/X_valid_preproc.npy", X_va)
np.save("datasets/X_test_preproc.npy", X_te)

In [24]:
# Save model
joblib.dump(gb_cal, "models/models/gb_cal_model.pkl")
cp.model.save("models/models/nn_model.keras")

In [25]:
# Save predictions
gb_train = gb_cal.predict_proba(X_tr_st)[:,1]
gb_valid = gb_cal.predict_proba(X_va_st)[:,1]
gb_test = gb_cal.predict_proba(X_te_st)[:,1]
train_df = pd.DataFrame({
    "y_train": y_train_full,
    "nn_train": nn_tr,
    "gb_train": gb_train
})
valid_df = pd.DataFrame({
    "y_valid": y_valid,
    "nn_valid": nn_va,
    "gb_valid": gb_valid
})
test_df = pd.DataFrame({
    "y_test": y_test,
    "nn_test": nn_te,
    "gb_test": gb_test
})

train_df.to_csv("datasets/preds_train.csv", index=False)
valid_df.to_csv("datasets/preds_valid.csv", index=False)
test_df.to_csv("datasets/preds_test.csv", index=False)

In [27]:
# Save churn probability with indices
X_te_df = pd.DataFrame(X_te, columns=feature_names, index=X_test.index)
p_churn = model.predict(X_te).reshape(-1)
df_plot = X_te_df.assign(p_churn=p_churn)
df_plot.to_csv("datasets/df_plot.csv", index=True)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [33]:
# Save feature names
remap = {"gender_Male":"Gender", "Dependents_Yes":"Dependents", "PhoneService_Yes":"PhoneService", "MultipleLines_Yes":"MultipleLines", "InternetService_Fiber optic":"InternetService"}
feature_names = [remap.get(f,f) for f in feature_names]
with open("datasets/feature_names.json", 'w') as f:
    json.dump(feature_names, f, indent=2)